### CNN Training - Train directly
Tbd: Summary what happens in this notebook

importing intern

In [4]:
from models.resnet_model import Hyperspectral_Resnet
from batchers import dataset_constants, batcher
from utils.run import get_full_experiment_name, make_log_and_ckpt_dirs
from utils.trainer import RegressionTrainer

importing extern

In [5]:
import os
import pickle
from pprint import pprint
import numpy as np
import tensorflow as tf

set root directory

In [6]:
# edit if necessary
# ROOT_DIR = 'C:/Users/matte/Documents/Data/01_Universitaet/02_TH_Koeln/06_Semester/
# 04_Machine_Learning_Project/CNN_Architecture'

define `run_training`

In [9]:
def run_training(sess, ooc, batcher_type, dataset, keep_frac, model_name, model_params, batch_size,
                 ls_bands, nl_band, label_name, augment, learning_rate, lr_decay,
                 max_epochs, print_every, eval_every, num_threads, cache, log_dir, save_ckpt_dir,
                 init_ckpt_dir, imagenet_weights_path, hs_weight_init, exclude_final_layer):
    '''
    Args
    - sess: tf.Session
    - ooc: bool, whether to use out-of-country split
    - batcher_type: str, type of batcher, one of ['base', 'urban', 'rural']
    - dataset: str, options depends on batcher_type
    - keep_frac: float
    - model_name: str, one of ['resnet', 'vggf', 'simplecnn', 'resnetcombo']
    - model_params: dict
    - batch_size: int
    - ls_bands: one of [None, 'rgb', 'ms']
    - nl_band: one of [None, 'merge', 'split']
    - label_name: str, name of the label in the TFRecord file
    - augment: bool
    - learning_rate: float
    - lr_decay: float
    - max_epochs: int
    - print_every: int
    - eval_every: int
    - num_threads: int
    - cache: list of str
    - log_dir: str, path to directory to save logs for TensorBoard, must already exist
    - save_ckpt_dir: str, path to checkpoint dir for saving weights
        - intermediate dirs must already exist
    - init_ckpt_dir: str, path to checkpoint dir from which to load existing weights
        - set to empty string '' to use ImageNet or random initialization
    - imagenet_weights_path: str, path to pre-trained weights from ImageNet
        - set to empty string '' to use saved ckpt or random initialization
    - hs_weight_init: str, one of [None, 'random', 'same', 'samescaled']
    - exclude_final_layer: bool, or None
    '''

    # set model class
    if model_name == 'resnet':
        model_class = Hyperspectral_Resnet
    # import model classes if needed
    elif model_name == 'vggf':
        model_class = VGGF
    elif model_name == 'simplecnn':
        model_class = SimpleCNN
    elif model_name == 'resnetcombo':
        model_class = ResnetCombo
    else:
        raise ValueError('Unknown model_name. Was not one of ["resnet", "vggf", "simplecnn", "resnetcombo"]'
                         'or model was not imported.')
    # check if paths exist
    assert os.path.exists(log_dir)
    assert os.path.exists(os.path.dirname(save_ckpt_dir))